# Viewing and manipulating data from FITS tables

## Authors
Lia Corrales, Kris Stern

## Learning Goals
* Download a FITS table file from a URL 
* Open a FITS table file and view table contents
* Make a 2D histogram with the table data
* Close the FITS file after use

## Keywords
FITS, file input/output, table, numpy, matplotlib, histogram


## Summary

This tutorial demonstrates the use of `astropy.utils.data` to download a data file, then uses `astropy.io.fits` and `astropy.table` to open the file. Lastly, `matplotlib` is used to visualize the data as a histogram.

In [ ]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from matplotlib.colors import LogNorm

# Set up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

The following line is needed to download the example FITS files used in this tutorial.

In [ ]:
from astropy.utils.data import download_file

FITS files often contain large amounts of multi-dimensional data and tables.  

In this particular example, we'll open a FITS file from a Chandra observation of the Galactic Center.  The file contains a list of events with x and y coordinates, energy, and various other pieces of information.

In [ ]:
event_filename = download_file('http://data.astropy.org/tutorials/FITS-tables/chandra_events.fits', 
                               cache=True)

## Opening the FITS file and viewing table contents

Since the file is big, let's open it with `memmap=True` to prevent RAM storage issues.

In [ ]:
hdu_list = fits.open(event_filename, memmap=True)

In [ ]:
hdu_list.info()

In this case, we're interested in reading EVENTS, which contains information about each X-ray photon that hit the detector.

To find out what information the table contains, let's print the column names.

In [ ]:
print(hdu_list[1].columns)

Now we'll take this data and convert it into an [astropy table](http://docs.astropy.org/en/stable/table/). While it's possible to access FITS tables directly from the ``.data`` attribute, using [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table) tends to make a variety of common tasks more convenient.

In [ ]:
evt_data = Table(hdu_list[1].data)

For example, a preview of the table is easily viewed by simply running a cell with the table as the last line:

In [ ]:
evt_data

We can extract data from the table by referencing the column name. Let's try making a histogram for the energy of each photon, which will give us a sense for the spectrum (folded with the detector's efficiency).

In [ ]:
energy_hist = plt.hist(evt_data['energy'], bins='auto')

## Making a 2D histogram with some table data

We'll make an image by binning the x and y coordinates of the events into a 2D histogram.

This particular observation spans five CCD chips.  First, we determine the events that only fell on the main (ACIS-I) chips, which have number ids 0, 1, 2, and 3.

In [ ]:
ii = np.in1d(evt_data['ccd_id'], [0, 1, 2, 3])
np.sum(ii)

### Method 1: Use numpy to make a 2D histogram and imshow to display it

This method allows us to create an image without stretching:

In [ ]:
NBINS = (100,100)

img_zero, yedges, xedges = np.histogram2d(evt_data['x'][ii], evt_data['y'][ii], NBINS)

extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

plt.imshow(img_zero, extent=extent, interpolation='nearest', cmap='gist_yarg', origin='lower')

plt.xlabel('x')
plt.ylabel('y')

# To see more color maps
# http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps

### Method 2: Use hist2d with a log-normal color scheme

In [ ]:
NBINS = (100,100)
img_zero_mpl = plt.hist2d(evt_data['x'][ii], evt_data['y'][ii], NBINS, 
                          cmap='viridis', norm=LogNorm())

cbar = plt.colorbar(ticks=[1.0,3.0,6.0])
cbar.ax.set_yticklabels(['1','3','6'])

plt.xlabel('x')
plt.ylabel('y')

## Close the FITS file

When you're done using a FITS file, it's often a good idea to close it.  That way you can be sure it won't continue using up excess memory or file handles on your computer.  (This happens automatically when you close Python, but you never know how long that might be...)

In [ ]:
hdu_list.close()

## Exercises

Make a scatter plot of the same data you histogrammed above.  The [plt.scatter](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter) function is your friend for this.  What are the pros and cons of doing it this way?

Try the same with the [plt.hexbin](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hexbin) plotting function. Which do you think looks better for this kind of data?

Choose an energy range to make a slice of the FITS table, then plot it. How does the image change with different energy ranges?